In [1]:
import os
import sys
import pyxrt
import contextlib
from neuroring import NeuroRingHost, NeuroRingKernel
import nest
import network
import numpy as np
from network_params import net_dict
from sim_params import sim_dict
from stimulus_params import stim_dict
from utils_binding import Options  # Ensure this module provides .index and .bitstreamFile

net = network.Network(sim_dict, net_dict, stim_dict)
net.create()
net.connect()

print(net.pops)

XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 2128708
UID: 1031
[Wed Aug  6 12:22:06 2025 GMT]
HOST: cyberdeck
EXE: /usr/bin/orted
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 2123860
UID: 1031
[Wed Aug  6 12:22:07 2025 GMT]
HOST


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.8.0
 Built: Jan 27 2025 08:18:03

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



/home/miahafiz/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning:Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
/home/miahafiz/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning:Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
/home/miahafiz/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning:Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
/home/miahafiz/NeuroRing/host_py/network.py:210: UserWarning:DC input created to compensate missing Poisson input.



Data will be written to: /home/miahafiz/NeuroRing/host_py/data/
  Directory already existed. Old data will be overwritten.

Neuron numbers are scaled by a factor of 0.010.
Indegrees are scaled by a factor of 0.010.
  Weights and DC input are adjusted to compensate.

RNG seed: 55
Total number of virtual processes: 1
Creating neuronal populations.

Aug 06 14:22:09 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Creating recording devices.
  Creating spike recorders.
Creating DC generators for external stimulation.
Connecting neuronal populations recurrently.
Connecting DC generators.

Aug 06 14:22:10 NodeManager::prepare_nodes [Info]: 
    Preparing 787 nodes for simulation.
[NodeCollection(metadata=None, model=iaf_psc_exp, size=207, first=1, last=207), NodeCollection(metadata=None, model=iaf_psc_exp, size=58, first=208, last=265), NodeCollection(metadata=None, model=iaf_psc_exp, size=219, first=266, last=484), NodeCollection(metadata=None, mode

In [2]:
host = NeuroRingHost(net, 4, 1, "/home/miahafiz/NeuroRing/krnl_neuroring_hw.xclbin")
host.initialize_devices()

Extracting synapse information...
Initialized device 0 with XCLBIN UUID: f34a489e-6e2a-1cb5-751e-f92b197776bf
threshold: -50.0
membrane_potential: -65.0
dcstim_amp: 209.11304111722728
{'simulation_time': 1, 'threshold': 3259498496, 'membrane_potential': 3263299584, 'amount_of_cores': 4, 'neuron_start': 1, 'neuron_total': 192, 'dcstim_start': 0, 'dcstim_total': 1000, 'dcstim_amp': 1129389296, 'device': None, 'xclbin': None, 'uuid': None, 'kernel_name': None, 'kernel': None}
Initialized kernel NeuroRing:{NeuroRing_0} and AxonLoader:{AxonLoader_0} on device <pyxrt.device object at 0x7fe0e8411970>
threshold: -50.0
membrane_potential: -65.0
dcstim_amp: 209.11304111722728
{'simulation_time': 1, 'threshold': 3259498496, 'membrane_potential': 3263299584, 'amount_of_cores': 4, 'neuron_start': 193, 'neuron_total': 193, 'dcstim_start': 0, 'dcstim_total': 1000, 'dcstim_amp': 1129389296, 'device': None, 'xclbin': None, 'uuid': None, 'kernel_name': None, 'kernel': None}
Initialized kernel NeuroRing:

In [6]:
import time

start_time = time.time()
host.run_kernels(10000)
host.wait_for_kernels(10000)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

write done kernel <pyxrt.kernel object at 0x7fe0a43e37f0>
Running kernel <pyxrt.kernel object at 0x7fe0a43e37f0>
Running kernel <pyxrt.kernel object at 0x7fe0a1df1830>
write done kernel <pyxrt.kernel object at 0x7fe0a43df4b0>
Running kernel <pyxrt.kernel object at 0x7fe0a43df4b0>
Running kernel <pyxrt.kernel object at 0x7fe0f34fe830>
write done kernel <pyxrt.kernel object at 0x7fe0a18e6430>
Running kernel <pyxrt.kernel object at 0x7fe0a18e6430>
Running kernel <pyxrt.kernel object at 0x7fe0a18e63f0>
write done kernel <pyxrt.kernel object at 0x7fe0a42934f0>
Running kernel <pyxrt.kernel object at 0x7fe0a42934f0>
Running kernel <pyxrt.kernel object at 0x7fe0a18e65f0>
Axon loader run complete <pyxrt.kernel object at 0x7fe0a43e37f0>
Kernel run complete <pyxrt.kernel object at 0x7fe0a1df1830>
Axon loader run complete <pyxrt.kernel object at 0x7fe0a43df4b0>
Kernel run complete <pyxrt.kernel object at 0x7fe0f34fe830>
Axon loader run complete <pyxrt.kernel object at 0x7fe0a18e6430>
Kernel run co

In [5]:
array = host.kernels_per_fpga[0][0].get_spike_recorder_array(1000)
print(array)
print(len(array))

for i in range(len(array)):
    if array[i] != 0:
        print(i)

[0 0 0 ... 0 0 0]
64000
128
144
160
176
192
208
224
240
256
272
288
304
320
336
352
368
384
400
416
432
448
464
480
496
512
528
544
560
576
592
608
624
640
656
672
688
704
720
736
752
768
784
800
816
832
848
864
880
896
912
928
944
960
976
992
1008
1024
1040
1056
1072
1088
1104
1120
1136
1152
1168
1184
1200
1216
1232
1248
1264
1280
1296
1312
1328
1344
1360
1376
1392
1408
1424
1440
1456
1472
1488
1504
1520
1536
1552
1568
1584
1600
1616
1632
1648
1664
1680
1696
1712
1728
1744
1760
1776
1792
1808
1824
1840
1856
1872
1888
1904
1920
1936
1952
1968
1984
2000
2016
2032
2048
2064
2080
2096
2112
2128
2144
2160
2176
2192
2208
2224
2240
2256
2272
2288
2304
2320
2336
2352
2368
2384
2400
2416
2432
2448
2464
2480
2496
2512
2528
2544
2560
2576
2592
2608
2624
2640
2656
2672
2688
2704
2720
2736
2752
2768
2784
2800
2816
2832
2848
2864
2880
2896
2912
2928
2944
2960
2976
2992
3008
3024
3040
3056
3072
3088
3104
3120
3136
3152
3168
3184
3200
3216
3232
3248
3264
3280
3296
3312
3328
3344
3360
3376
3392
3408
3

In [ ]:
print(len(host.packed_list_per_fpga[0][0]))
print(len(host.packed_list_per_fpga[0][1]))


In [ ]:
host.kernels_per_fpga[0][0].run_kernel(host.packed_list_per_fpga[0][0], 2)
host.kernels_per_fpga[0][1].run_kernel(host.packed_list_per_fpga[0][1], 2)

In [ ]:
host.kernels_per_fpga[0][0].wait_for_axon()

In [ ]:
host.kernels_per_fpga[0][1].wait_for_axon()

In [ ]:
host.kernels_per_fpga[0][0].wait_for_neuroring()

In [ ]:
host.kernels_per_fpga[0][1].wait_for_neuroring()

In [ ]:
host.kernels_per_fpga[0][0].wait_for_kernel()
print(host.kernels[0].spikeRecorder_array)


In [ ]:
for i in range(len(host.kernels[0].spikeRecorder_array)):
    if host.kernels[0].spikeRecorder_array[i] > 0:
        print(i)

In [ ]:
print(host.kernels[0].spikeRecorder_array)


In [ ]:
print(host.kernel_neuron_ranges)

In [ ]:
print(net.DC_amp)
print(np.average(net.DC_amp))

In [ ]:
print(host.kernels_per_fpga[0][1].kernel.group_id(0))
print(host.kernels_per_fpga[0][1].kernel.group_id(1))
print(host.kernels_per_fpga[0][1].kernel.group_id(2))
print(host.kernels_per_fpga[0][1].kernel.group_id(3))
print(host.kernels_per_fpga[0][1].kernel.group_id(4))
print(host.kernels_per_fpga[0][1].kernel.group_id(5))
print(host.kernels_per_fpga[0][1].kernel.group_id(6))
print(host.kernels_per_fpga[0][1].kernel.group_id(7))
print(host.kernels_per_fpga[0][1].kernel.group_id(8))
print(host.kernels_per_fpga[0][1].kernel.group_id(9))
print(host.kernels_per_fpga[0][1].kernel.group_id(10))
print(host.kernels_per_fpga[0][1].kernel.group_id(11))
print(host.kernels_per_fpga[0][1].kernel.group_id(12))
print(host.kernels_per_fpga[0][1].kernel.group_id(13))
print(host.kernels_per_fpga[0][1].kernel.group_id(14))
print(host.kernels_per_fpga[0][1].kernel.group_id(15))
print(host.kernels_per_fpga[0][1].kernel.group_id(16))
print(host.kernels_per_fpga[0][1].kernel.group_id(17))
